In [1]:
import pandas as pd

# https://files.grouplens.org/datasets/movielens/ml-25m.zip
# Using pandas to open 
movies = pd.read_csv("movies.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'movies.csv'

In [ ]:
movies.head()

In [ ]:
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

In [ ]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [ ]:
movies

NameError: name 'movies' is not defined

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

NameError: name 'movies' is not defined

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]
    
    return results

In [5]:
# pip install ipywidgets
#jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [6]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

ModuleNotFoundError: No module named 'ipywidgets'

In [7]:
movie_id = 89745

#def find_similar_movies(movie_id):
movie = movies[movies["movieId"] == movie_id]

NameError: name 'movies' is not defined

In [8]:
ratings = pd.read_csv("ratings.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'ratings.csv'

In [9]:
ratings.dtypes

NameError: name 'ratings' is not defined

In [10]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

NameError: name 'ratings' is not defined

In [11]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

NameError: name 'ratings' is not defined

In [12]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]

NameError: name 'similar_user_recs' is not defined

In [13]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

NameError: name 'ratings' is not defined

In [14]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

NameError: name 'all_users' is not defined

In [15]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

NameError: name 'similar_user_recs' is not defined

In [16]:
rec_percentages

NameError: name 'rec_percentages' is not defined

In [17]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

NameError: name 'rec_percentages' is not defined

In [18]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

NameError: name 'rec_percentages' is not defined

In [19]:
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")


NameError: name 'rec_percentages' is not defined

In [20]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [21]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

ModuleNotFoundError: No module named 'ipywidgets'